In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-large')

model = T5ForConditionalGeneration.from_pretrained('t5-large')

# input = "My name is Azeem and I live in India"
# input = "It has never been easy to have a rational conversation about the value of gold."
input = "The horse rides the astronaut. "
# You can also use "translate English to French" and "translate English to Romanian"
input_ids = tokenizer("translate English to German: "+input, return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded)

In [15]:
## Convert SGM to txt file
import re
path='./sgm/newstest2019-ende-src.en.sgm'#'./sgm/newstest2019-ende-ref.de.sgm'#'./sgm/newstest2019-deen-ref.en.sgm'
text_out='newstest2019-ende-src.txt'#'newstest2019-ende-ref.txt'#'newstest2019-deen-ref.txt'
seg_re = re.compile(r"<seg id=\"\d+\">(.*)</seg>")
with open(text_out,'w', encoding="utf-8") as g:
    with open(path, encoding="utf-8") as f:
        for line in f:
            seg_match = re.match(seg_re, line)
            if seg_match:
                assert len(seg_match.groups()) == 1
            else:
                continue
            out_line= seg_match.groups()[0]
            g.write(out_line)
            g.write('\n')

In [62]:
## Translate txt from de to en
# from transformers import FSMTForConditionalGeneration, FSMTTokenizer, 
from transformers import MarianMTModel, MarianTokenizer
mname = 'Helsinki-NLP/opus-mt-de-en'#"facebook/wmt19-en-de"#"facebook/wmt19-de-en"
# tokenizer = FSMTTokenizer.from_pretrained(mname)
# model = FSMTForConditionalGeneration.from_pretrained(mname)

text_out='newstest2019-deen-in.txt'
import torch
with torch.no_grad():
    model=MarianMTModel.from_pretrained(mname).to('cuda')
tokenizer=MarianTokenizer.from_pretrained(mname)
fair_out_path = 'newstest2019-deen-helsinki_out.txt'
with open(text_out, encoding="utf-8") as f:
    lines = f.readlines()
with open(fair_out_path,'w', encoding="utf-8") as g:
    for input in lines:
        input_ids = tokenizer.encode(input.strip(), return_tensors="pt").to('cuda')
        outputs = model.generate(input_ids)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # print(decoded)
        g.write(decoded)
        g.write('\n')
        #  # Machine learning is great, isn't it?

In [77]:
## evaluate the output
from sacrebleu import corpus_bleu
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    print('reslts:',result)
    with open(score_path,"w") as score_file:
        score_file.write(result)
all_paths=['newstest2019-deen-helsinki_out.txt','SD_out_deen_v5_alltags_beam10_SAT3.out',\
    'SD_out_deen_v4_lemma_beam5_SAT2_prune100.out','SD_out_deen_v4.out', \
        'newstest2019-deen-out.txt', 'SD_out_deen_neurologic_deocding.out' ]        
for item in all_paths:
    print(item)
    out_path =item
    #'SD_out_deen_neurologic_deocding.out' ,  
    with open(out_path) as output:
        output_lns=output.readlines()[:1000]
    print(len(output_lns))
    with open('newstest2019-deen-ref.txt') as reflns:
        refs_lns=reflns.readlines()[:1000]
    #('newstest2019-deen-out.txt','newstest2019-deen-ref.txt') -> 41.17
    #('newstest2019-deen-out.txt','newstest2019-deen-ref.txt')  -> 39.26
    calculate_bleu_score(output_lns, refs_lns, './blueScore_%s'%out_path.split('.')[:-1])

        # output_lns1=[]
        # refs_lns1=[]
#         for i,line1 in enumerate(output_lns):
#             line2 = refs_lns[i]
# #     if len(line2.split('.')) > 2:
# #         print(i)
# #         continue
# #     # if line1.split() < line2.split():
# #     #     continue
#         output_lns1.append(line1)
#         refs_lns1.append(line2)
    # print('data length is:',i+1)   
    # print('limited data length is:',len(output_lns1))       
    # calculate_bleu_score(output_lns1, refs_lns1, './blueScore_SD_5')
    
# newstest2019-deen-helsinki_out.txt
# reslts: BLEU score: 39.508582858290566
# SD_out_deen_v3.out
# reslts: BLEU score: 34.84768488447905
# SD_out_deen_v4.out
# reslts: BLEU score: 34.718709113680596
# newstest2019-deen-out.txt
# reslts: BLEU score: 41.16915939988038
# SD_out_deen_neurologic_deocding.out
# reslts: BLEU score: 39.62539180524859


newstest2019-deen-helsinki_out.txt
720
reslts: BLEU score: 40.08618133755379
SD_out_deen_v5_alltags_beam10_SAT3.out
640
reslts: BLEU score: 38.36683188368516
SD_out_deen_v4_lemma_beam5_SAT2_prune100.out
720
reslts: BLEU score: 38.22663389109872
SD_out_deen_v4.out
128
reslts: BLEU score: 34.718709113680596
newstest2019-deen-out.txt
720
reslts: BLEU score: 40.48188167528294
SD_out_deen_neurologic_deocding.out
720
reslts: BLEU score: 40.06757138362206


In [38]:
## extract structures
import stanza
from typing import List
from stanza_batch import batch
import json
from nltk.corpus import stopwords
import pickle
path1='newstest2019-deen-in.txt'
# path1='/home/mbastan/context_home/structuralDecoding/a_star_neurologic/dataset/machine_translation/newstest2017-iate/iate.414.terminology.tsv.de'

pkname='newstest2019'
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
nlp = stanza.Pipeline('de', processors='lemma,tokenize,pos,depparse',use_gpu=True)
with open(path1) as f:
    lines = f.readlines()
try:
    with open('stanza_documents_%s_de.pk'%pkname,'rb') as f:
        stanza_documents=pickle.load(f)
except:
    ## batch document processing to increase the speed
    stanza_documents: List[stanza.Document] = []
    for document in batch(lines, nlp, batch_size=32): # Default batch size is 32
        stanza_documents.append(document)
    ## save the documents on a file
    with open('stanza_documents_%s_de.pk'%pkname,'wb') as f:
        pickle.dump(stanza_documents,f)  


/home/mbastan/anaconda3/envs/hug3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-20 01:02:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-20 01:02:01 WARNING: Language de package default expects mwt, which has been added
2023-01-20 01:02:02 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2023-01-20 01:02:02 INFO: Use device: gpu
2023-01-20 01:02:02 INFO: Loading: tokenize
2023-01-20 01:02:03 INFO: Loading: mwt
2023-01-20 01:02:03 INFO: Loading: pos
2023-01-20 01:02:04 I

In [61]:
# from deep_translator import PonsTranslator
# translator = PonsTranslator(source='de', target='en')
nlp_en = stanza.Pipeline('en', processors='lemma,tokenize,pos,depparse',use_gpu=True,\
    depparse_model_path='/home/mbastan/context_home/structuralDecoding/stanza_model/en_outall_parser.pt')
# exit()

# from deep_translator import GoogleTranslator
# from deep_translator import LingueeTranslator
from googletrans import Translator

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# translator = LingueeTranslator(source='de', target='en')
translator = Translator()


outpath='newstest2019_deen_en_v4_lemma.constraint.json' #'iate.414_en.terminology.constraint.json'
with open(outpath,'w') as f:
    stop_words = set(stopwords.words('english'))
    german_stop_words=set(stopwords.words('german'))
    
    indd = 0
    for doc in  stanza_documents:
        indd += 1
        if indd % 100==0:
            print('processing %d'%indd)
        sent = doc.sentences[0]
        arr_all = []
        arr_ori = []
        arr_rev = []
        arr_lex =[]
        sent_map ={}
        for word in sent.words:
                # print('word is',word.text)
                if not any(c.isalpha() for c in word.text):
                    continue
                arr_all.append([word.text,word.deprel])
                if not (word.deprel == 'nsubj' or word.deprel == 'obj' or word.deprel == 'root' or word.deprel == 'obl'):
                    continue
                translated_words=[]
                try:
                    # translated_words=translator.translate(word.text,return_all=True)
                    tr=translator.translate(src='de', dest='en',text=word.text)
                    # print('tr is',tr)
                    translated_words.append(tr.text)
                    if tr.extra_data['all-translations']:
                        translated_words.extend(tr.extra_data['all-translations'][0][1])
                    # print('translated_words',translated_words)
                except Exception as e:
                    print('error e:',e)
                    translated_words=[word.text]
                # for translated_word in translated_words:
                translated_nlp=nlp_en('\n'.join(item for item in translated_words))
                translated_lemma=[]
                for translated_sent in translated_nlp.sentences:
                    for w in translated_sent.words:
                        if w.lemma:
                            curr_w=w.lemma
                        else:
                            curr_w=w.text
                        if not(curr_w in stop_words or curr_w.lower() in stop_words):
                            translated_lemma.append(curr_w)
                            translated_lemma.append(curr_w.lower())
                # print('translated_lemma',translated_lemma)             
                for tlemma in set(translated_lemma):  
                    arr_ori.append([tlemma,word.deprel])
                    
        json.dump(arr_ori,f)
        f.write('\n')
        f.flush()

2023-01-20 01:52:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-20 01:52:15 INFO: Loading these models for language: en (English):
| Processor | Package                 |
---------------------------------------
| tokenize  | combined                |
| pos       | combined                |
| lemma     | combined                |
| depparse  | /home/mbas..._parser.pt |

2023-01-20 01:52:15 INFO: Use device: gpu
2023-01-20 01:52:15 INFO: Loading: tokenize
2023-01-20 01:52:15 INFO: Loading: pos
2023-01-20 01:52:15 INFO: Loading: lemma
2023-01-20 01:52:15 INFO: Loading: depparse
2023-01-20 01:52:16 INFO: Done loading processors!


processing 100
processing 200
processing 300
processing 400
processing 500
processing 600
processing 700
processing 800
processing 900
processing 1000
processing 1100
processing 1200
processing 1300
processing 1400
processing 1500
processing 1600
processing 1700
processing 1800
processing 1900
processing 2000


In [59]:
# # nlp_en('\n'.join(item for item in translated_words))
# # translated_words
# # translator.translate(word.text)
# from deep_translator import LingueeTranslator, GoogleTranslator

# word = 'good'

# GoogleTranslator(source='de', target='en').translate('Münchnerin')

# # word
# tr.text()
# word
# translator.translate(src='de', dest='en',text='Schöne').text
tr.text

'Munich woman'

In [47]:
from googletrans import Translator

translator = Translator()
word='werden'
st=translator.translate(text='Münchnerin',src='de',dest='en')#.extra_data['all-translations'])
st.extra_data['all-translations']


In [32]:
from deep_translator import PonsTranslator
translator=PonsTranslator('de','en')
input_text='"Schöne Münchnerin" 2018 werden!'
docc = nlp(input_text)
input_text=docc.sentences[0]
for word in input_text.words:
    print(word.deprel)
    print(word.text)
    try:
        tt=translator.translate(word.text,return_all=True)
        print('all transltions',tt)
        translated_nlp=nlp_en(tt)
        # break
        for translated_sent in translated_nlp.sentences:
                for w in translated_sent.words:
                    print('translated_nlp',w.text)
                    # print('lemma:',w.lemma)
                    # print('in stop words',w.lemma in stop_words)
    except Exception as e:
        print('nashod: error:',e)
        print(word.text)

punct
"
nashod: error: Request exception can happen due to an api connection error. Please check your connection and try again
"
amod
Schöne
all transltions ['beauty ', 'Beauty and the Beast ', 'to look after sth ', 'to take care of sth ', 'to be kind to sth ', 'to go easy on sth ', "to be kinder to one's liver ", "to have to go bit easy on one's health heart ", 'it is easy on the joints ', 'to be kind to sth ', 'this detergent is kind to the fabric your hands ', 'to spare sb sth ', "to spare sb's feelings ", 'to spare sb ', 'to take things easy ', 'Scania ', 'to embellish sth ', 'to dress sth up ', 'to refine or clear sth ', 'are we there yet ', 'have you heard ', "they're coming today ", 'you want to leave now or already ', 'was finished after only five minutes ', 'was already finished after five minutes ', 'very soon after ', 'it is already late or late already ', "just one minute and then I'm gone ", 'hardly have time to lie down before the telephone rings yet again ', 'she had har

In [59]:
# translator.translate('warden')
from deep_translator import LingueeTranslator
# import translators as ts
# word = 'werden'
# ts.bing(word,from_language='de',to_language='en',professional_field='common')
translated_word = LingueeTranslator(source='de', target='en').translate(word,return_all=True)
translated_word

['become',
 'go',
 'get',
 'turn out',
 'be',
 'being',
 'existence',
 'essence',
 'his',
 'its']

In [24]:
german_stop_words=set(stopwords.words('german'))
translated_lemma=[]
for word in sent.words:
    if not any(c.isalpha() for c in word.text):
        continue
    print(word.text.lower())
    print(word.text.lower()  in german_stop_words)
    translated_word=translator.translate(word.text) 
    print('translation:',translated_word)
    print(translated_word in stop_words)
    translated_nlp=nlp(translated_word)
    for translated_sent in translated_nlp.sentences:
        for w in translated_sent.words:
            print('w lemma:',w.lemma)
            print(' w.lemma.lower() not in stop_words:', w.lemma.lower() not in stop_words)
            print('w.text.lower() not in stop_words:',w.text.lower() not in stop_words)
    # translated_lemma.append()
    translated_lemma = [w.lemma for translated_sent in translated_nlp.sentences \
                    for w in translated_sent.words\
                    if w.lemma.lower() not in stop_words and w.text.lower() not in stop_words ]
    print('translated_lemma:',translated_lemma)

schöne
False
translation: Lovely
False
w lemma: Lovely
 w.lemma.lower() not in stop_words: True
w.text.lower() not in stop_words: True
translated_lemma: ['Lovely']
münchnerin
False
translation: Munich woman
False
w lemma: Munich
 w.lemma.lower() not in stop_words: True
w.text.lower() not in stop_words: True
w lemma: woman
 w.lemma.lower() not in stop_words: True
w.text.lower() not in stop_words: True
translated_lemma: ['Munich', 'woman']
werden
True
translation: will
True
w lemma: wollen
 w.lemma.lower() not in stop_words: True
w.text.lower() not in stop_words: False
translated_lemma: []


In [5]:
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
translator = GoogleTranslator(source='de', target='en')
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [74]:
### compare translation outputs

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def cosine_similarity(X,Y):
        # tokenization
        X_list = word_tokenize(X) 
        Y_list = word_tokenize(Y)
        
        # sw contains the list of stopwords
        sw = stopwords.words('english') 
        l1 =[];l2 =[]
        
        # remove stop words from the string
        X_set = {w for w in X_list if not w in sw} 
        Y_set = {w for w in Y_list if not w in sw}
        
        # form a set containing keywords of both strings 
        rvector = X_set.union(Y_set) 
        for w in rvector:
            if w in X_set: l1.append(1) # create a vector
            else: l1.append(0)
            if w in Y_set: l2.append(1)
            else: l2.append(0)
        c = 0
        
        # cosine formula 
        for i in range(len(rvector)):
                c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        # print("similarity: ", cosine)
        
        return(cosine)
        
constraints=open('newstest2019_deen_en_v2_lemma.constraint.json').readlines()
sd_out=open('SD_out_deen_v5_alltags_beam10_SAT3.out').readlines()
nd_out=open('SD_out_deen_neurologic_deocding.out').readlines()
ops_out=open('newstest2019-deen-helsinki_out.txt').readlines()
ref=open('newstest2019-deen-ref.txt').readlines()

import pandas as pd

rows=[]
for i,(cons,sd,nd,ops,re) in enumerate(zip(constraints,sd_out,nd_out,ops_out,ref)):
    sd_tg_cosine=cosine_similarity(sd,re)
    nd_tg_cosine=cosine_similarity(nd,re)
    if sd == nd:
        continue
    row={
        'ind':i,
        'constrains':cons,
        'structura_decoding':sd,
        'neurological_decoding':nd,
        'ops':ops,
        'target':re,
        'sd_tg_cosine':sd_tg_cosine,
        'nd_tg_cosine':nd_tg_cosine,
        'sd_nd_diff':sd_tg_cosine-nd_tg_cosine
    }
    rows.append(row)
    if i==100:
        break
    
df=pd.DataFrame(rows)
df.to_csv('translation_out_100.csv',index=False)

In [9]:
examples=$HOME/.mt-metrics-eval/mt-metrics-eval/wmt20/metric-scores/en-de


,ind,constrains,structura_decoding,neurological_decoding,fairseq,target
0,0,"[[""Munich"", ""root""]]\n",Beautiful Munich girl 2018: Beautiful Munich w...,Beautiful Munich girl 2018: Beautiful Munich w...,Beautiful Munich 2018: Beautiful Munich 2018 i...,The Beauty of Munich 2018: the Beauty of Munic...
1,1,"[[""www.ecs-webhosting.de"", ""root""], [""04.05.20...","By az, updated on 04.\n","By az, updated on 04.05.2018 at 11:11\n","By az, updated on 04 / 05 / 2018 at 11: 11\n","From A-Z, updated on 04/05/2018 at 11:11\n"
2,2,"[[""she"", ""nsubj""], [""to"", ""root""], [""want"", ""r...","Yeah, she wants...\n","Yeah, she wants...\n","Yes, she wants...\n","Yes, she wants to...\n"
3,3,"[[""2018"", ""obl""], [""he's"", ""root""], [""just"", ""...","""Beautiful Munich"" 2018 will be!\n","""Beautiful Munich"" 2018 will be!\n","Become a ""beautiful Munich woman"" in 2018!\n","to become ""The Beauty of Munich"" in 2018!\n"
4,4,"[[""afternoon"", ""obl""], [""to"", ""root""], [""wait""...","In the afternoon, another surprise awaits our ...","In the afternoon, another surprise awaits our ...",In the afternoon another surprise awaits our c...,In the afternoon there is another surprise wai...


In [73]:
nlp_en('John introduced Kayla to Tim.')

[
  [
    {
      "id": 1,
      "text": "John",
      "lemma": "John",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 2,
      "deprel": "nsubj",
      "start_char": 0,
      "end_char": 4
    },
    {
      "id": 2,
      "text": "introduced",
      "lemma": "introduce",
      "upos": "VERB",
      "xpos": "VBD",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "start_char": 5,
      "end_char": 15
    },
    {
      "id": 3,
      "text": "Kayla",
      "lemma": "Kayla",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 2,
      "deprel": "obj",
      "start_char": 16,
      "end_char": 21
    },
    {
      "id": 4,
      "text": "to",
      "lemma": "to",
      "upos": "ADP",
      "xpos": "IN",
      "head": 5,
      "deprel": "case",
      "start_char": 22,
      "end_char": 24
    },
    {
      "id": 5,
      "text": "Tim",
      